In [1]:
import numpy as np
import pandas as pd
import os
import time
import psutil as pu
from datetime import datetime
import matplotlib.animation as animation
from matplotlib import pyplot as plt
import pyRAPL
pyRAPL.setup() 

In [6]:
def pretty_print(mem):
    for start in ['', 'K', 'M', 'G', 'T', 'P']:
        if mem < 1024:
            return f"{mem:.2f}{start}B"
        mem /= 1024

In [3]:
def info_of_process():
    #Empty list to store all process data 
    process_infos = []
    
    for proc in pu.process_iter():
        proc_info = dict()
        #Suggested in documentation to speed up working
        
        meter = pyRAPL.Measurement('bar')
        meter.begin()
        
        with proc.oneshot():
            process_id = -100
            if proc.pid == 0:
                continue
            else:
                process_id = proc.pid
                
            proc_info["pid"] = process_id
            proc_info["ppid"] = proc.ppid()
            proc_info["name"] = proc.name()
            proc_info["cpu_usage_percent"] = proc.cpu_percent()
            proc_info["status"] = proc.status()
            proc_info["number_of_threads"] = proc.num_threads()
            
            try:
                time_of_create = proc.create_time()
                time_create = datetime.fromtimestamp(time_of_create)
            except OSError:
                time_of_create = pu.boot_time()
                time_create = datetime.fromtimestamp(time_of_create)
                
            try:
                cores = len(proc.cpu_affinity())
            except pu.AccessDenied:
                cores = 0
            
            try:
                nice_priority = int(proc.nice())
            except pu.AccessDenied:
                nice_priority = 0
                
            try:
                memory_usage = proc.memory_full_info().uss
            except pu.AccessDenied:
                memory_usage = 0
                
            try:
                username = proc.username()
            except psutil.AccessDenied:
                username = "Not run at Sudo"
                
            wait_cat = ''
            if proc.status() == 'idle':
                if proc.cpu_times().iowait > 0.0:
                    wait_cat = 'I/O'
                else:
                    wait_cat = 'CPU_WAIT'
            else:
                wait_cat = 'N/A'
                
            proc_info["creation_time"] = time_create
            proc_info["Cores_of_CPU"] = cores
            proc_info["Nice_Priority"] = nice_priority
            proc_info["Memory_Used"] = memory_usage
            proc_info["UserName"] = username
            proc_info["Wait_Bound"] = wait_cat
            meter.end()
            proc_info["Power_Consumption"] = meter.result.dram
            
        
        process_infos.append(proc_info)
        
    return process_infos

In [4]:
process_infos = info_of_process()

In [7]:
# print(process_infos)
# type(process_infos)
df = pd.DataFrame(process_infos)
df['Memory_Used'] = df['Memory_Used'].apply(pretty_print)
df

,pid,ppid,name,cpu_usage_percent,status,number_of_threads,creation_time,Cores_of_CPU,Nice_Priority,Memory_Used,UserName,Wait_Bound,Power_Consumption
0,1,0,systemd,0.0,sleeping,1,2021-11-16 00:32:31.020,8,0,0.00B,root,N/A,[2319.0]
1,2,0,kthreadd,0.0,sleeping,1,2021-11-16 00:32:31.020,8,0,0.00B,root,N/A,[1038.0]
2,4,2,kworker/0:0H,0.0,idle,1,2021-11-16 00:32:31.020,1,-20,0.00B,root,CPU_WAIT,[366.0]
3,6,2,mm_percpu_wq,0.0,idle,1,2021-11-16 00:32:31.020,8,-20,0.00B,root,CPU_WAIT,[427.0]
4,7,2,ksoftirqd/0,0.0,sleeping,1,2021-11-16 00:32:31.020,1,0,0.00B,root,N/A,[488.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,29731,1119,apache2,0.0,sleeping,27,2021-11-17 01:06:53.500,8,0,0.00B,www-data,N/A,[0.0]
404,29732,1119,apache2,0.0,sleeping,27,2021-11-17 01:06:53.500,8,0,0.00B,www-data,N/A,[428.0]
405,30799,1838,gvfsd-network,0.0,sleeping,4,2021-11-17 01:28:31.780,8,0,1.18MB,manav,N/A,[244.0]
406,30826,1838,gvfsd-dnssd,0.0,sleeping,3,2021-11-17 01:28:37.540,8,0,1.21MB,manav,N/A,[244.0]


In [11]:
cust_cols = ["pid","name"]
len(cust_cols)
# cust_cols[0]

2

In [13]:
# gives a single float value
pu.cpu_percent()
# gives an object with many fields
pu.virtual_memory()
# you can convert that object to a dictionary 
dict(pu.virtual_memory()._asdict())
# you can have the percentage of used RAM
print(pu.virtual_memory().percent)

# you can calculate percentage of available memory
print(pu.virtual_memory().available * 100 / pu.virtual_memory().total)


72.1
27.85306643994949


In [18]:
%matplotlib notebook

y = []
frame_len = 200
def animate(i):
    y.append(pu.cpu_percent())
    
    if len(y)<= frame_len:
        plt.cla()
        plt.plot(y)
    
    plt.tight_layout()

ani = animation.FuncAnimation(plt.gcf(), animate, interval=1000)

<IPython.core.display.Javascript object>